# Code Generator

Use a frontier model to generate high performance C++ code from python code

In [1]:
# imports
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [2]:
# environment

load_dotenv(override=True)
openai_key = os.environ['OPENAI_API_KEY']
anthropic_key = os.environ['ANTHROPIC_API_KEY']

In [3]:
# initialize

openai = OpenAI()
claude = anthropic.Anthropic()
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with the C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [14]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```", "")
    with open("./my-work/optimized.cpp", "w") as f:
        f.write(code)

In [15]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [16]:
def optimize_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply=""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end='', flush=True)
    write_output(reply)

In [17]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution time: {(end_time - start_time):.6f} seconds")
"""

In [18]:
# exec runs python code found in string - be careful
exec(pi)

Result: 3.141592658589
Execution time: 12.296533 seconds


In [19]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::chrono::duration<double> execution_time = end_time - start_time;
    std::cout << std::fixed << std::setprecision(12)
              << "Result: " << result << std::endl;
    std::cout << "Execution time: " << std::fixed << std::setprecision(6)
              << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [20]:
exec(pi)

Result: 3.141592658589
Execution time: 12.180042 seconds


# Compiling C++ and executing

This next cell contains the command to compile a C++ file on my M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In the next lab (day4), a student has contributed a full solution that compiles to efficient code on Mac, PC and Linux!

You can wait for this, or you can google (or ask ChatGPT!) for how to do this on your platform, then replace the lines below.
If you're not comfortable with this step, you can skip it for sure - I'll show you exactly how it performs on my Mac.


OR alternatively: student Sandeep K.G. points out that you can run Python and C++ code online to test it out that way. Thank you Sandeep!  
> Not an exact comparison but you can still get the idea of performance difference.
> For example here: https://www.programiz.com/cpp-programming/online-compiler/

In [23]:
# Compile C++ and run the executable
!clang++ -03 -std=c++17 -march=armv8.3-a -o optimized.cpp 
!./optimized

/bin/bash: line 1: clang++: command not found
/bin/bash: line 1: ./optimized: No such file or directory


In [24]:
optimize_claude(pi)

#include <iostream>
#include <iomanip>
#include <chrono>

double calculate(long long iterations, int param1, int param2) {
    double result = 1.0;
    #pragma omp parallel for reduction(-:result)
    for (long long i = 1; i <= iterations; ++i) {
        double j = i * static_cast<double>(param1) - param2;
        result -= (1.0 / j);
        j = i * static_cast<double>(param1) + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    double result = calculate(100'000'000, 4, 1) * 4;
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;

    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time);
    std::cout << "Execution time: " << duration.count() / 1e6 << " seconds" << std::endl;

    return 0;
}

In [25]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
!./optimized

/bin/bash: line 1: clang++: command not found
/bin/bash: line 1: ./optimized: No such file or directory


In [26]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [27]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 40.766225 seconds


In [28]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>

// Linear Congruential Generator
class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 4294967296)
        : value(seed), a(a), c(c), m(m) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t value, a, c, m;
};

// Generates max subarray sum
int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = (lcg.next() % (max_val - min_val + 1)) + min_val;
    }
    
    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
     

***# skipping compiling c++, but c++ gpt creates throws error***

In [32]:
optimize_claude(python_hard)

#include <iostream>
#include <vector>
#include <chrono>
#include <limits>
#include <iomanip>

using namespace std;
using namespace chrono;

uint32_t lcg(uint32_t& seed, const uint32_t a = 1664525, const uint32_t c = 1013904223, const uint32_t m = 1UL << 32) {
    seed = (a * seed + c) & 0xFFFFFFFF;
    return seed;
}

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg(seed) % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = numeric_limits<int64_t>::min();
    int64_t current_sum = 0;
    int64_t min_seen = 0;

    for (int num : random_numbers) {
        current_sum += num;
        max_sum = max(max_sum, current_sum - min_seen);
        min_seen = min(min_seen, current_sum);
    }

    return max_sum;
}

int64_t total_max_subarray_sum(int n, uint32_t initial_seed, int min_val, int max_val) {
    int64_t total_sum = 0;
    uint32_t seed = initial_se

***skipping compiling c++, but claude code does not throw error***

## Now to create similar with gradio interface

In [33]:
def stream_gpt(python):
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=messages_for(python),
        stream=True
    )
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n', '').replace('```', '')

In [34]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```cpp\n', '').replace('```', '')

In [35]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    else:
        raise ValueError("Unknown Model")
    for stream_so_far in result:
        yield stream_so_far

In [36]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python Code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ Code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select Model", value="GPT")
        convert = gr.Button("Convert Code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [37]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [43]:
# reworked via Claude

def execute_cpp(code):
    write_output(code)

    try:

        # Try different compilers that might be available
        compilers = ["g++", "clang++", "c++"]
        compiler_found = False

        for compiler in compilers:

            try:
                # Try to compile C++ code with current compiler
                compile_cmd = [compiler, "-Ofast", "-std=c++17", "-o", "optimized", "./my-work/optimized.cpp"]
                print(f"Trying to compile with: {compiler}")
                compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
                compiler_found = True
                print(f"Compilation successful with {compiler}")
                break

            except FileNotFoundError:
                print(f"{compiler} not found, trying next compiler...")
            except subprocess.CalledProcessError as e:
                print(f"Compilation with {compiler} failed: {e.stderr}")
                continue

        if not compiler_found:
            print("No C++ compiler found. Please install g++ or clang++.")
            return None

        # Run the compiled program
        run_cmd = ["./optimized"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return None


In [44]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [45]:
# gradio interface

with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch()